# Stock Price Prediction Using LSTM

In [88]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

In [89]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [90]:
company ='FB'
start = dt.datetime(2012,1,1)
end = dt.datetime(2020,1,1)

## Load The Data

In [91]:
data = web.DataReader(company, 'yahoo', start, end)

In [92]:
data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-05-18,45.000000,38.000000,42.049999,38.230000,573576400,38.230000
2012-05-21,36.660000,33.000000,36.529999,34.029999,168192700,34.029999
2012-05-22,33.590000,30.940001,32.610001,31.000000,101786600,31.000000
2012-05-23,32.500000,31.360001,31.370001,32.000000,73600000,32.000000
2012-05-24,33.209999,31.770000,32.950001,33.029999,50237200,33.029999


## Prepare The Data

In [93]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

In [94]:
prediction_days = 60

In [95]:
X_train = []
Y_train = []

In [96]:
for x in range(prediction_days, len(scaled_data)):
    X_train.append(scaled_data[x-prediction_days:x,0])
    Y_train.append(scaled_data[x,0])

In [97]:
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [98]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

## Build the Model

In [99]:
model = Sequential()
model.add(LSTM(units=50,return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [100]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [101]:
model.fit(X_train, Y_train, epochs=25, batch_size=32)

Epoch 1/25
59/59 [==============================] - 3s 46ms/step - loss: 0.0497
Epoch 2/25
59/59 [==============================] - 3s 44ms/step - loss: 0.0179
Epoch 3/25
59/59 [==============================] - 3s 43ms/step - loss: 0.0137
Epoch 4/25
59/59 [==============================] - 2s 42ms/step - loss: 0.0115
Epoch 5/25
59/59 [==============================] - 2s 41ms/step - loss: 0.0100
Epoch 6/25
59/59 [==============================] - 2s 41ms/step - loss: 0.0088
Epoch 7/25
59/59 [==============================] - 2s 41ms/step - loss: 0.0082
Epoch 8/25
59/59 [==============================] - 2s 41ms/step - loss: 0.0074
Epoch 9/25
59/59 [==============================] - 2s 41ms/step - loss: 0.0075
Epoch 10/25
59/59 [==============================] - 2s 42ms/step - loss: 0.0065
Epoch 11/25
59/59 [==============================] - 2s 41ms/step - loss: 0.0062
Epoch 12/25
59/59 [==============================] - 2s 41ms/step - loss: 0.0061
Epoch 13/25
59/59 [==================

  ## Test the Model Accuracy on Existing Data

In [135]:
test_start = dt.datetime(2020,1,1)
test_end = dt.datetime.now()

In [136]:
test_data = web.DataReader(company, 'yahoo',test_start,test_end)
test_data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-12-31,205.559998,203.600006,204.000000,205.250000,8953500,205.250000
2020-01-02,209.789993,206.270004,206.750000,209.779999,12077100,209.779999
2020-01-03,210.399994,206.949997,207.210007,208.669998,11188400,208.669998
2020-01-06,212.779999,206.520004,206.699997,212.600006,17058900,212.600006
2020-01-07,214.580002,211.750000,212.820007,213.059998,14912400,213.059998


In [137]:
actual_prices = test_data['Close'].values

In [138]:
total_dataset=pd.concat((data['Close'],test_data['Close']),axis=0)

In [139]:
model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values

In [140]:
model_inputs = model_inputs.reshape(-1,1)

In [141]:
model_inputs = scaler.transform(model_inputs)

## Make Prediction on test data

In [142]:
x_test = []

for X in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[X-prediction_days:X,0])


In [143]:
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [147]:
predicted_prices = model.predict(x_test)
predicted_prices

array([[[0.7412479 ],
        [0.78736746],
        [0.79247   ],
        ...,
        [0.88180304],
        [0.87731206],
        [0.87506986]],

       [[0.73225725],
        [0.7941555 ],
        [0.7939048 ],
        ...,
        [0.877311  ],
        [0.8750688 ],
        [0.8740556 ]],

       [[0.7420418 ],
        [0.79523087],
        [0.80567014],
        ...,
        [0.8750676 ],
        [0.87405443],
        [0.8799608 ]],

       ...,

       [[1.151057  ],
        [1.0433534 ],
        [1.0556043 ],
        ...,
        [0.9973041 ],
        [1.0033165 ],
        [0.9967574 ]],

       [[1.1423779 ],
        [1.0585761 ],
        [1.1067214 ],
        ...,
        [1.0033193 ],
        [0.9967598 ],
        [0.98486364]],

       [[1.1619035 ],
        [1.134112  ],
        [1.1206524 ],
        ...,
        [0.99676263],
        [0.984866  ],
        [0.9710183 ]]], dtype=float32)

In [148]:
plt.plot(actual_prices, color="black")
plt.plot(predicted_prices , color="green")
plt.title(f"{company} Shape Price")
plt.xlabel("Time")
plt.ylabel(f"{Comapany} Share Price")
plt.legend()
plt.show()


ValueError: x and y can be no greater than 2-D, but have shapes (274,) and (274, 60, 1)

# Predict Next Day

In [149]:
real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs+1),0]]

In [150]:
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1],1))

In [153]:
prediction = model.predict(real_data)
print(f"Prediction: {prediction}")

Prediction: [[[1.3051783 ]
  [1.1208848 ]
  [1.0967513 ]
  [1.0784489 ]
  [1.0574282 ]
  [1.0353905 ]
  [1.0194259 ]
  [1.009683  ]
  [0.99996436]
  [0.9911301 ]
  [0.9867295 ]
  [0.98225236]
  [0.9788908 ]
  [0.9850303 ]
  [0.98901916]
  [0.9931903 ]
  [0.99473965]
  [1.0037543 ]
  [1.0100965 ]
  [1.0079132 ]
  [1.0034257 ]
  [1.0052153 ]
  [1.0048937 ]
  [0.9997239 ]
  [0.9953729 ]
  [0.9899596 ]
  [0.9873705 ]
  [0.9877267 ]
  [0.9887613 ]
  [0.9886488 ]
  [0.99026775]
  [0.9883133 ]
  [0.9818485 ]
  [0.9783399 ]
  [0.9762393 ]
  [0.9841498 ]
  [0.9899174 ]
  [0.9885348 ]
  [0.9878504 ]
  [0.98363924]
  [0.9824307 ]
  [0.97548807]
  [0.9754865 ]
  [0.9756422 ]
  [0.9666265 ]
  [0.9551803 ]
  [0.94857836]
  [0.9406314 ]
  [0.941512  ]
  [0.95364225]
  [0.9688264 ]
  [0.9826971 ]
  [0.9909725 ]
  [0.9973272 ]
  [1.0033376 ]
  [0.99677587]
  [0.9848776 ]
  [0.9710283 ]
  [0.9654764 ]]]
